In [2]:
# 再次导入numpy和matplotlib.pyplot
import numpy as np
import matplotlib.pyplot as plt

# 从 scikit-learn 库中导入线性回归模型
# sklearn.linear_model 模块包含了各种线性模型
from sklearn.linear_model import LinearRegression

# 从 scikit-learn 库中导入均方误差评估指标
# sklearn.metrics 模块包含了各种模型评估工具
from sklearn.metrics import mean_squared_error

# 从 scipy.stats 导入 shapiro 函数用于正态性检验
from scipy.stats import shapiro

# 导入tqdm库，用于显示一个漂亮的进度条，方便我们知道实验进行到哪里了。
# 这在进行耗时较长的循环时非常有用！
from tqdm import tqdm
# --- 1. 定义数据生成函数 ---

def generate_data(n_samples=100):
    """
    生成非线性数据。
    我们创建一个带有噪声的正弦波数据。
    """
    # np.random.rand(n_samples) 生成 n_samples 个 [0, 1) 之间的均匀分布的数。
    # 乘以4，将范围扩展到 [0, 4)。
    x = np.sort(4 * np.random.rand(n_samples))
    
    # 真实的函数关系是一个正弦波
    y_true = np.sin(2 * np.pi * x / 4)
    
    # 为真实值添加一些服从正态分布的噪声，让数据更真实
    # np.random.normal(mean, std_dev, size)
    # mean=0: 噪声均值为0
    # std_dev=0.2: 噪声的标准差为0.2
    # size=n_samples: 噪声的数量和x,y一致
    noise = np.random.normal(0, 0.2, n_samples)
    y_noisy = y_true + noise
    
    # scikit-learn中的模型要求输入X是二维数组，格式为 [n_samples, n_features]。
    # 我们的x现在是一维的 [n_samples,]，需要转换。
    # x.reshape(-1, 1) 中的-1表示“自动计算该维度的大小”，1表示我们有1个特征。
    return x.reshape(-1, 1), y_noisy


# --- 2. 进行多次实验，收集泛化误差 ---

# 设置实验参数
n_experiments = 1000  # 我们要重复进行1000次实验
generalization_errors = [] # 创建一个空列表，用于存储每一次实验的泛化误差

# 使用tqdm来包装我们的循环，这样就会有一个进度条
print("Running experiments to collect generalization errors...")
for _ in tqdm(range(n_experiments)):
    # 在每次循环中，都生成全新的训练数据和测试数据
    # 这是关键！测试数据必须是模型在训练过程中“从未见过”的。
    X_train, y_train = generate_data(n_samples=50) # 50个点的训练集
    X_test, y_test = generate_data(n_samples=50)   # 50个点的测试集

    # a. 创建并训练模型
    # 创建一个线性回归模型的实例
    model = LinearRegression()
    # 使用 .fit() 方法来训练模型
    # fit(X, y) 会找到一条直线，使得这条直线到所有训练数据点(X_train, y_train)的距离平方和最小。
    model.fit(X_train, y_train)

    # b. 在测试集上进行预测
    # 使用 .predict() 方法，传入新的数据X_test，模型会给出预测的y值。
    y_pred = model.predict(X_test)
    
    # c. 计算并存储泛化误差
    # mean_squared_error(y_true, y_pred) 计算均方误差
    # 公式为：MSE = (1/n) * Σ(y_true_i - y_pred_i)^2
    # 它衡量了预测值和真实值之间差异的平均平方。
    error = mean_squared_error(y_test, y_pred)
    generalization_errors.append(error)

print("Experiments finished.")


# --- 3. 可视化其中一次实验的结果，以理解“欠拟合” ---

print("Visualizing a single case of underfitting...")
# 为了可视化，我们再生成一次数据
X_train_vis, y_train_vis = generate_data(n_samples=50)
# 训练一个新模型
model_vis = LinearRegression()
model_vis.fit(X_train_vis, y_train_vis)
# 生成一条平滑的线用于绘制真实的函数曲线
x_smooth = np.linspace(0, 4, 200).reshape(-1, 1)
y_true_smooth = np.sin(2 * np.pi * x_smooth / 4)
y_fit_line = model_vis.predict(x_smooth)

plt.figure(figsize=(10, 6))
# 绘制散点图，代表我们的训练数据
plt.scatter(X_train_vis, y_train_vis, alpha=0.6, label='Training Data')
# 绘制真实的函数曲线
plt.plot(x_smooth, y_true_smooth, 'g--', linewidth=2, label='True Function (Sine Wave)')
# 绘制模型学习到的线性拟合线
plt.plot(x_smooth, y_fit_line, 'r-', linewidth=2, label='Linear Fit (Underfitting)')
plt.title('Demonstration of Underfitting', fontsize=16)
plt.xlabel('Feature (X)', fontsize=12)
plt.ylabel('Target (y)', fontsize=12)
plt.legend()
plt.grid(True)
plt.show()


# --- 4. 分析泛化误差的分布 ---

print("Analyzing the distribution of generalization errors...")
# a. 绘制泛化误差的直方图
plt.figure(figsize=(10, 6))
plt.hist(generalization_errors, bins=30, density=True, alpha=0.7, label='Distribution of Gen. Errors')
plt.title('Histogram of Generalization Errors', fontsize=16)
plt.xlabel('Mean Squared Error', fontsize=12)
plt.ylabel('Density', fontsize=12)
plt.grid(True)
plt.show()

# b. 进行Shapiro-Wilk正态性检验
# shapiro()函数接收一个数据样本，返回一个包含两个值的元组：
# 1. 检验统计量 (statistic): W值
# 2. p-value
stat, p_value = shapiro(generalization_errors)

print("\n--- Shapiro-Wilk Normality Test Results ---")
print(f"Test Statistic: {stat:.4f}")
print(f"P-value: {p_value:.4f}")

# 根据p值给出结论
alpha = 0.05 # 显著性水平
if p_value > alpha:
    print(f"P-value ({p_value:.4f}) is greater than {alpha}. We fail to reject the null hypothesis.")
    print("Conclusion: The generalization errors appear to be normally distributed.")
else:
    print(f"P-value ({p_value:.4f}) is less than or equal to {alpha}. We reject the null hypothesis.")
    print("Conclusion: The generalization errors do not appear to be normally distributed.")

Running experiments to collect generalization errors...


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 6243.47it/s]


Experiments finished.
Visualizing a single case of underfitting...


<Figure size 1000x600 with 1 Axes>

Analyzing the distribution of generalization errors...


<Figure size 1000x600 with 1 Axes>


--- Shapiro-Wilk Normality Test Results ---
Test Statistic: 0.9918
P-value: 0.0000
P-value (0.0000) is less than or equal to 0.05. We reject the null hypothesis.
Conclusion: The generalization errors do not appear to be normally distributed.
